In [1]:
from benchmarking.analysis.analyze_results import *

In [2]:
# impls = ["threaded", "asyncio", "vanilla"]
impls = ["threaded", "asyncio"]
libs = ["torch", "lightning"]

In [3]:
def extract_metadata(
    output_base_folder: Path, folder_filter: str = "**", filter_by_metadata: Dict[str, List[str]] = None,
):
    files = list(output_base_folder.rglob(f"{folder_filter}/results-*.log"))
    data = []
    for working_file_path in tqdm.tqdm(files, total=len(files)):
        results = parse_results_log(working_file_path)
        if len(results) == 0:
            continue
        with (working_file_path.parent / "metadata.json").open("r") as f:
            metadata = json.load(f)
        if filter_by_metadata is not None:
            for k, v in filter_by_metadata.items():
                if metadata[k] not in v:
                    continue
        results = pd.DataFrame.from_records(data=results)

        for k, v in metadata.items():
            if not isinstance(v, (int, float, complex)):
                results[k] = str(v)
            else:
                results[k] = v

        results["source_file"] = working_file_path
        results["run"] = working_file_path.parent.name

        # filter out old data format missing dataset etc.
        if "dataset" not in results.columns:
            continue

        data.append(results)
    df = pd.concat(data)
    df.groupby
    return df

In [4]:
def get_returns_data(folder_filter="*e2e*"):
    output_base_folder = Path("/iarai/home/moritz.neun/git/storage-benchmarking/benchmark_output/")
    df_dataloader_s3 = extract_timelines(output_base_folder, folder_filter=folder_filter)
    
    df_details_s3 = extract_metadata(output_base_folder, folder_filter=folder_filter)
    
    # extract unique functions (for s3 and scratch, it's the same)
    unique_functions_s3 = np.unique(df_dataloader_s3["item_x"])

    # define colors for timelane items 
    colors = {}
    for i, color in zip(unique_functions_s3, ["red", "gold", "blue", "magenta", "aqua"]):
        colors[str(i)] = color
    
    # define lanes (each lane is for a single function call (i.e. usage))
    lanes={}
    for i, lane in zip(unique_functions_s3, range(len(unique_functions_s3))):
        lanes[str(i)] = lane

    # print to verify
    print(f"Unique functions: {unique_functions_s3}")
    print(f"Lanes: {lanes}")
    
    # get the names of the runs for the iterators later on 
    unique_runs_s3 = np.unique(df_dataloader_s3["run"])
    print(f"Runs: {unique_runs_s3}")

    df_gpuutil_s3 = extract_gpuutil(output_base_folder, folder_filter=folder_filter)
    
    returns_s3 = []
    for run in sorted(unique_runs_s3):
        df = df_dataloader_s3[df_dataloader_s3["run"]==run]
        dfgpu = df_gpuutil_s3[df_gpuutil_s3["run"]==run]
        r = show_timelines_with_gpu(df, dfgpu, lanes, colors, run, False, True, False, 2)
        r["dataset_limit"] = df_details_s3[df_details_s3["run"]==run].get("dataset_limit").iloc[0]
        r["epochs"] = df_details_s3[df_details_s3["run"]==run].get("epochs").iloc[0]
        returns_s3.append(r)
    return returns_s3

def plot_img_per_s(returns_data):
    results = pd.DataFrame.from_records(data=returns_data)
    results["throughput"] = results["dataset_limit"] / (results["runtime"] / results["epochs"])
    results.sort_values(["runtime", "library"], ascending=True).round(2)
    fig, ax = plt.subplots(figsize=(20, 10))
    labels = []
    for epochs in np.unique(results["epochs"]):
        data = results[(results["epochs"]==epochs) & (results["implementation"] != "vanilla")]
        y = list(data["throughput"])
        x = range(len(y))
        points = list(zip(data["implementation"], data["library"]))
        print(points)
        ax.plot(x, y, label=f"epochs: {epochs}", marker='o', linewidth=2, markersize=12)
        for i, txt in enumerate(points):
            print(txt)
            ax.annotate(txt, (x[i], y[i]))
    ax.set_xlabel("Experiment index", loc="center")
    ax.set_ylabel("Throughput (img/s)", loc="center")
    ax.set_ylim(0.0, 400.0)
    ax.grid(linestyle="--", which="both")
    ax.set_title("S3 access, different number of epochs")
    ax.legend()

In [5]:
returns_data = get_returns_data("e2e_calibration/*")

  4%|█████████                                                                                                                                                                                                                                 | 41/1064 [00:03<01:17, 13.20it/s]


KeyboardInterrupt: 

In [ ]:
plot_img_per_s(returns_data)

In [ ]:
returns_data_120 = get_returns_data("e2e_calibration_120/*")

In [ ]:
plot_img_per_s(returns_data_120)